In [1]:
print("hi")

hi


In [45]:
# this does a full permutation shuffle at every iteration
import csv
%matplotlib inline
import matplotlib.pyplot as plt
#import pandas as pd
import sys        
from scipy import stats as scistats
from random import shuffle

def get_cps(seq):
    cps = 0
    num_pairs = 0
    for i in range(0,(len(seq)-6),3):
        num_pairs += 1
        cps += cp_scores[seq[i:i+6]]
    return(cps)

def get_cpb(seq):
    cps = 0
    num_pairs = 0
    for i in range(0,(len(seq)-6),3):
        num_pairs += 1
        cps += cp_scores[seq[i:i+6]]
    #print(num_pairs)
    #print(cps)
    return((cps/num_pairs) )

# codon pair score of any length sequence
def get_cps_indep(seq):
    cps_indep = 0 
    # for indep_dependent pairs scores - frame 2
    for i in range(1,(len(seq)-3),3):
        if (seq[i:i+6] not in cp_scores_indep):
            #print("Missing " + seq[i:i+6] + " from independent scores")
            continue
        cps_indep += cp_scores_indep[seq[i:i+6]]
    # for indep_dependent pairs scores - frame 3
    for i in range(2,(len(seq)-3),3):
        if (seq[i:i+6] not in cp_scores_indep):
            #print("Missing " + seq[i:i+6] + " from independent scores")
            continue
        cps_indep += cp_scores_indep[seq[i:i+6]]
    return(cps_indep)

# codon adaptation index of any length sequence
def get_cai(seq):
    # get the CAI as the geometric mean of all the codon's individual CAI values
    cai_list = []
    for i in range(0,len(seq),3):
        cai_list.append(codon_CAI[seq[i:(i+3)]])
    return(scistats.gmean(cai_list))

# read in synonymous codons
aa_positions = dict()
aa_to_syn_codons_dict = dict()
codon_to_aa_dict = dict()
csvfile = open('AA_CODONS_MAP')
reader = csv.reader(csvfile, delimiter='\t')
for row in reader:
    mylist = row[3].split(',')
    aa_to_syn_codons_dict[row[0]] = mylist
    aa_positions[row[0]] = []
    for codon in mylist:
        codon_to_aa_dict[codon] = row[0]

# read in the human orfs and make the CAI for all the codons
codon_occs = dict()
# set up the dict
for codon in (codon_to_aa_dict.keys()):
    codon_occs[codon] = 0
genes_file = open("human_CCDS_nucleotide.current.fna")
for line in genes_file:
    line = line.rstrip()
    if line[0] != ">": 
        if (len(line)%3 != 0):
            #print("PROBLEM, genes file is not multiple of 3 per line.")
            continue
        # get the frequency of each codon
        for i in range(0,len(line),3):
            codon_occs[line[i:(i+3)]] += 1

# get the CAI of each codon
codon_CAI = dict()
for aa in (aa_to_syn_codons_dict.keys()):
    highest_frequency = 0
    for codon in (aa_to_syn_codons_dict[aa]):
        if (codon_occs[codon] > highest_frequency):
            highest_frequency = codon_occs[codon]
    for codon in (aa_to_syn_codons_dict[aa]):
        if (highest_frequency == 0):
            codon_CAI[codon] = 0
        else:
            codon_CAI[codon] = codon_occs[codon]/highest_frequency
            
cp_scores = dict()
cp_scores_file = open("human_cps_coleman.csv")
cp_reader = csv.reader(cp_scores_file, delimiter='\t')
for line in cp_reader:
    cp_scores[line[1]] = float(line[5])


cp_scores_indep = dict()
cp_scores_file = open("human_FIndep_Scores.csv")
cp_reader = csv.reader(cp_scores_file, delimiter=',')
for line in cp_reader:
    cp_scores_indep[line[0]] = float(line[1])
            
            
# read in the gene's codon sequence
codons_file = open("polio_genomics.fsa")
original_codons = ""
for line in codons_file:
    line = line.rstrip()
    if line[0] != ">":
        original_codons = original_codons + line
codons_file.close()

original_AA = ""
for i in range(0,len(original_codons),3):
    original_AA += codon_to_aa_dict[original_codons[i:i+3]]
    
recoded_sequence = "GGTGCTCACGAAAACTCGAATCGCGCGTACGGCTCGTCGACGATCAACTACACGACGATCAACTACTATCGCGATTCGGCGTCGAACGCGGCGTCGAAACAAGACTTCTCGCAAGACCCGTCGAAATTTACCGAACCGATTAAAGATGTTTTAATAAAAACGGCGCCGATGTTAAATTCGCCGAACATAGAAGCTTGCGGTTATTCGGATCGCGTATTACAACTTACGCTGGGCAACTCGACGATCACCACGCAAGAAGCGGCGAACTCGGTCGTCGCGTACGGTCGATGGCCAGAATATTTACGCGACAGTGAGGCGAATCCGGTCGATCAACCAACCGAACCGGACGTCGCGGCGTGCCGATTTTATACGCTCGATACGGTCTCCTGGACGAAAGAATCTCGAGGCTGGTGGTGGAAACTACCGGACGCGTTACGCGACATGGGCCTCTTTGGACAAAACATGTACTACCACTACTTGGGTCGATCCGGTTATACCGTACATGTTCAGTGCAACGCGTCGAAATTTCATCAAGGTGCTTTGGGTGTTTTCGCGGTACCGGAGATGTGCCTGGCGGGGGACTCGAATACGACGACGATGCACACGTCGTACCAAAACGCGAACCCAGGCGAAAAGGGGGGCACCTTCACGGGCACCTTCACGCCGGACAACAACCAGACGTCGCCGGCGCGACGATTCTGCCCGGTCGATTATTTACTTGGAAACGGTACGCTGCTGGGGAACGCGTTCGTATTTCCTCATCAAATAATAAATTTACGAACCAACAATTGCGCGACGCTTGTTTTACCGTACGTCAACAGCCTCTCGATCGACTCGATGGTGAAGCACAACAACTGGGGTATCGCGATCCTCCCGCTGGCGCCGTTAAATTTCGCGAGCGAATCGTCGCCGGAGATCCCGATTACGCTCACGATCGCGCCGATGTGTTGCGAATTTAATGGTTTACGAAATATTACGCTGCCGCGATTACAAGGACTACCGGTCATGAATACGCCGGGCTCGAACCAGTACCTCACGGCGGACAACTTCCAGAGCCCTTGCGCGTTACCGGAATTCGACGTTACGCCGCCGATCGATATTCCTGGAGAAGTAAAAAACATGATGGAGCTGGCGGAGATCGATACGATGATTCCTTTCGATCTTTCCGCGACGAAAAAAAATACGATGGAGATGTATCGCGTACGATTATCCGACAAGCCGCATACCGACGATCCGATCCTCTGCCTCTCGCTCTCGCCCGCGAGCGATCCGCGCCTCTCGCATACGATGCTTGGAGAAATATTAAATTATTATACGCACTGGGCGGGCAGCCTCAAGTTCACCTTCCTCTTCTGTGGCTCGATGATGGCGACGGGGAAATTACTTGTCTCGTACGCGCCGCCGGGGGCGGACCCGCCGAAAAAGCGAAAGGAGGCGATGCTTGGAACTCATGTCATCTGGGATATCGGTTTACAAAGCAGCTGCACCATGGTGGTGCCCTGGATCTCGAATACGACGTACCGCCAGACGATCGACGATTCCTTCACGGAGGGGGGCTACATCTCGGTCTTCTACCAAACGCGAATCGTCGTACCGCTCTCGACGCCGCGCGAAATGGACATTCTTGGTTTCGTATCGGCCTGCAATGACTTCTCGGTACGATTATTACGCGACACCACGCATATCGAACAAAAAGCGCTGGCGCAAGGACTTGGACAAATGCTGGAGTCGATGATCGACAACACGGTACGCGAAACGGTTGGTGCCGCGACGTCGCGCGACGCGTTACCGAATACGGAGGCGTCGGGGCCCACGCACTCGAAAGAAATACCGGCGCTCACGGCGGTCGAAACGGGCGCGACGAACCCGCTTGTTCCAAGCGATACCGTACAAACGCGACATGTTGTACAACATCGATCGCGAAGTGAATCGTCGATCGAAAGTTTTTTCGCGAGGGGGGCCTGTGTCACGATCATGACGGTCGACAACCCGGCGTCGACGACGAACAAAGATAAACTTTTCGCGGTCTGGAAGATCACGTACAAAGATACCGTACAATTACGACGAAAATTAGAATTCTTCACGTACTCGCGATTCGATATGGAGCTCACCTTTGTTGTTACCGCGAACTTCACGGAGACGAACAATGGACATGCGCTCAATCAAGTATATCAAATAATGTACGTACCGCCGGGGGCGCCGGTACCGGAGAAGTGGGATGACTACACGTGGCAGACGTCGTCGAACCCGTCGATCTTCTACACGTACGGTACCGCGCCGGCGCGAATATCGGTACCGTACGTGGGCATCTCGAACGCGTACTCGCACTTTTACGACGGCTTCTCGAAAGTTCCTTTAAAAGATCAATCGGCGGCGCTGGGGGACTCGTTATACGGGGCGGCGTCGCTCAACGATTTCGGTATTCTCGCGGTACGAGTAGTAAATGACCACAACCCGACGAAAGTCACGTCGAAAATTCGCGTATATTTAAAACCGAAACATATTCGCGTATGGTGCCCGCGGCCGCCGCGCGCGGTCGCGTATTACGGTCCTGGGGTCGATTATAAAGATGGTACGCTTACGCCGCTCTCGACGAAAGACCTCACGACGTAC"

back_to_original = ""
diff_total = 0
for i in range(0,len(recoded_sequence),3):
    if (recoded_sequence[i:i+3] != original_codons[i:i+3]):
        diff_total += 1
    back_to_original += codon_to_aa_dict[recoded_sequence[i:i+3]]
print("total number of codon diferences between original and recoded sequences " + str(diff_total))
if (original_AA == back_to_original):
    print("AA sequence remains the same, good")

print("coleman codon pair bias of the original sequence: " + str(get_cpb(original_codons)))
print("coleman codon pair bias of the recoded sequence: " + str(get_cpb(recoded_sequence)))
print("frame2&3 indep codon pair score of the original sequence: " + str(get_cps_indep(original_codons)))
print("frame2&3 indep codon pair score of the recoded sequence: " + str(get_cps_indep(recoded_sequence)))

# now, for each amino acid, find all of the positions to scramble
for i in range(0,len(recoded_sequence),3):
    aa = codon_to_aa_dict[recoded_sequence[i:i+3]]
    aa_positions[aa].append(i)

#print(aa_positions)
number_permutations = 1

best_so_far_sequence = recoded_sequence
best_so_far_indep_cps = get_cps_indep(recoded_sequence)
for p in range(0, number_permutations):
    aa_permuted_positions = dict()
    # permute all the postions
    for aa in (aa_to_syn_codons_dict.keys()):
        if (aa == "C"):
            print(aa)
        temp_list = list(aa_positions[aa])
        if (aa == "C"):
            print(temp_list)
        shuffle(temp_list)
        if (aa == "C"):
            print(temp_list)
        aa_permuted_positions[aa] = temp_list
    # construct the new string with all the permutations
    candidate_sequence = ""
    for i in range(0,len(recoded_sequence),3):
        aa = codon_to_aa_dict[recoded_sequence[i:i+3]]
        pos = aa_permuted_positions[aa].index(i)
        if (aa == "C"):
            print(i, " ", pos)
        candidate_sequence += recoded_sequence[pos:pos+3]
    candidate_cps = get_cps_indep(candidate_sequence)
    if(candidate_cps > best_so_far_indep_cps):
        best_so_far_sequence = candidate_sequence
        best_so_far_indep_cps = candidate_cps

back_to_recoded = ""
diff_total = 0
for i in range(0,len(recoded_sequence),3):
    if (best_so_far_sequence[i:i+3] != recoded_sequence[i:i+3]):
        diff_total += 1
    back_to_recoded += codon_to_aa_dict[best_so_far_sequence[i:i+3]]
if (original_AA == back_to_recoded):
    print("AA sequence remains the same, good")
else:
    print(original_AA)
    print(back_to_recoded)

print("coleman codon pair bias of the candidate sequence: " + str(get_cpb(candidate_sequence)))
print("frame2&3 indep codon pair score of the candidate sequence: " + str(get_cps_indep(candidate_sequence)))

total number of codon diferences between original and recoded sequences 576
AA sequence remains the same, good
coleman codon pair bias of the original sequence: -0.03513233601841192
coleman codon pair bias of the recoded sequence: -0.036011507479861975
frame2&3 indep codon pair score of the original sequence: -31.61352552200005
frame2&3 indep codon pair score of the recoded sequence: -488.30513615799987
C
[195, 357, 510, 567, 699, 789, 945, 948, 1059, 1242, 1353, 1482, 1641, 1962, 2514]
[1482, 1962, 567, 945, 789, 699, 510, 1242, 1353, 2514, 195, 948, 1641, 357, 1059]
195   10
357   13
510   6
567   2
699   5
789   4
945   3
948   11
1059   14
1242   7
1353   8
1482   0
1641   12
1962   1
2514   9
GAHENSNRAYGSSTINYTTINYYRDSASNAASKQDFSQDPSKFTEPIKDVLIKTAPMLNSPNIEACGYSDRVLQLTLGNSTITTQEAANSVVAYGRWPEYLRDSEANPVDQPTEPDVAACRFYTLDTVSWTKESRGWWWKLPDALRDMGLFGQNMYYHYLGRSGYTVHVQCNASKFHQGALGVFAVPEMCLAGDSNTTTMHTSYQNANPGEKGGTFTGTFTPDNNQTSPARRFCPVDYLLGNGTLLGNAFVFPHQIINLRTNNCATLVLPYVNSLSIDSMVKHNNWGIAILPL

In [122]:
# this does a greedy improvement algorithm - just swap one pair at a time, and only accept if the score increases
import csv
%matplotlib inline
import matplotlib.pyplot as plt
#import pandas as pd
import sys        
from scipy import stats as scistats
from random import shuffle
import random

def get_cps(seq):
    cps = 0
    num_pairs = 0
    for i in range(0,(len(seq)-6),3):
        num_pairs += 1
        cps += cp_scores[seq[i:i+6]]
    return(cps)

def get_cpb(seq):
    cps = 0
    num_pairs = 0
    for i in range(0,(len(seq)-6),3):
        num_pairs += 1
        cps += cp_scores[seq[i:i+6]]
    #print(num_pairs)
    #print(cps)
    return((cps/num_pairs) )

# codon pair score of any length sequence
def get_cps_indep(seq):
    cps_indep = 0 
    # for indep_dependent pairs scores - frame 2
    for i in range(1,(len(seq)-3),3):
        if (seq[i:i+6] not in cp_scores_indep):
            #print("Missing " + seq[i:i+6] + " from independent scores")
            continue
        cps_indep += cp_scores_indep[seq[i:i+6]]
    # for indep_dependent pairs scores - frame 3
    for i in range(2,(len(seq)-3),3):
        if (seq[i:i+6] not in cp_scores_indep):
            #print("Missing " + seq[i:i+6] + " from independent scores")
            continue
        cps_indep += cp_scores_indep[seq[i:i+6]]
    return(cps_indep)

# codon adaptation index of any length sequence
def get_cai(seq):
    # get the CAI as the geometric mean of all the codon's individual CAI values
    cai_list = []
    for i in range(0,len(seq),3):
        cai_list.append(codon_CAI[seq[i:(i+3)]])
    return(scistats.gmean(cai_list))

def get_codon_count_dict(seq):
    codon_count_dict = dict()
    for codon in (codon_to_aa_dict.keys()):
        codon_count_dict[codon] = 0
    for i in range(0,len(seq),3):
        codon_count_dict[seq[i:i+3]] += 1
    return codon_count_dict

# read in synonymous codons
aa_positions = dict()
aa_to_syn_codons_dict = dict()
codon_to_aa_dict = dict()
csvfile = open('AA_CODONS_MAP')
reader = csv.reader(csvfile, delimiter='\t')
for row in reader:
    mylist = row[3].split(',')
    aa_to_syn_codons_dict[row[0]] = mylist
    aa_positions[row[0]] = []
    for codon in mylist:
        codon_to_aa_dict[codon] = row[0]

# read in the human orfs and make the CAI for all the codons
codon_occs = dict()
# set up the dict
for codon in (codon_to_aa_dict.keys()):
    codon_occs[codon] = 0
genes_file = open("human_CCDS_nucleotide.current.fna")
for line in genes_file:
    line = line.rstrip()
    if line[0] != ">": 
        if (len(line)%3 != 0):
            #print("PROBLEM, genes file is not multiple of 3 per line.")
            continue
        # get the frequency of each codon
        for i in range(0,len(line),3):
            codon_occs[line[i:(i+3)]] += 1

# get the CAI of each codon
codon_CAI = dict()
for aa in (aa_to_syn_codons_dict.keys()):
    highest_frequency = 0
    for codon in (aa_to_syn_codons_dict[aa]):
        if (codon_occs[codon] > highest_frequency):
            highest_frequency = codon_occs[codon]
    for codon in (aa_to_syn_codons_dict[aa]):
        if (highest_frequency == 0):
            codon_CAI[codon] = 0
        else:
            codon_CAI[codon] = codon_occs[codon]/highest_frequency
            
            
cp_scores = dict()
cp_scores_file = open("human_cps_coleman.csv")
cp_reader = csv.reader(cp_scores_file, delimiter='\t')
for line in cp_reader:
    cp_scores[line[1]] = float(line[5])


cp_scores_indep = dict()
cp_scores_file = open("human_FIndep_Scores.csv")
cp_reader = csv.reader(cp_scores_file, delimiter=',')
for line in cp_reader:
    cp_scores_indep[line[0]] = float(line[1])
            
            
# read in the gene's codon sequence
codons_file = open("polio_genomics.fsa")
original_codons = ""
for line in codons_file:
    line = line.rstrip()
    if line[0] != ">":
        original_codons = original_codons + line
codons_file.close()

original_AA = ""
for i in range(0,len(original_codons),3):
    original_AA += codon_to_aa_dict[original_codons[i:i+3]]
    
recoded_sequence = "GGTGCTCACGAAAACTCGAATCGCGCGTACGGCTCGTCGACGATCAACTACACGACGATCAACTACTATCGCGATTCGGCGTCGAACGCGGCGTCGAAACAAGACTTCTCGCAAGACCCGTCGAAATTTACCGAACCGATTAAAGATGTTTTAATAAAAACGGCGCCGATGTTAAATTCGCCGAACATAGAAGCTTGCGGTTATTCGGATCGCGTATTACAACTTACGCTGGGCAACTCGACGATCACCACGCAAGAAGCGGCGAACTCGGTCGTCGCGTACGGTCGATGGCCAGAATATTTACGCGACAGTGAGGCGAATCCGGTCGATCAACCAACCGAACCGGACGTCGCGGCGTGCCGATTTTATACGCTCGATACGGTCTCCTGGACGAAAGAATCTCGAGGCTGGTGGTGGAAACTACCGGACGCGTTACGCGACATGGGCCTCTTTGGACAAAACATGTACTACCACTACTTGGGTCGATCCGGTTATACCGTACATGTTCAGTGCAACGCGTCGAAATTTCATCAAGGTGCTTTGGGTGTTTTCGCGGTACCGGAGATGTGCCTGGCGGGGGACTCGAATACGACGACGATGCACACGTCGTACCAAAACGCGAACCCAGGCGAAAAGGGGGGCACCTTCACGGGCACCTTCACGCCGGACAACAACCAGACGTCGCCGGCGCGACGATTCTGCCCGGTCGATTATTTACTTGGAAACGGTACGCTGCTGGGGAACGCGTTCGTATTTCCTCATCAAATAATAAATTTACGAACCAACAATTGCGCGACGCTTGTTTTACCGTACGTCAACAGCCTCTCGATCGACTCGATGGTGAAGCACAACAACTGGGGTATCGCGATCCTCCCGCTGGCGCCGTTAAATTTCGCGAGCGAATCGTCGCCGGAGATCCCGATTACGCTCACGATCGCGCCGATGTGTTGCGAATTTAATGGTTTACGAAATATTACGCTGCCGCGATTACAAGGACTACCGGTCATGAATACGCCGGGCTCGAACCAGTACCTCACGGCGGACAACTTCCAGAGCCCTTGCGCGTTACCGGAATTCGACGTTACGCCGCCGATCGATATTCCTGGAGAAGTAAAAAACATGATGGAGCTGGCGGAGATCGATACGATGATTCCTTTCGATCTTTCCGCGACGAAAAAAAATACGATGGAGATGTATCGCGTACGATTATCCGACAAGCCGCATACCGACGATCCGATCCTCTGCCTCTCGCTCTCGCCCGCGAGCGATCCGCGCCTCTCGCATACGATGCTTGGAGAAATATTAAATTATTATACGCACTGGGCGGGCAGCCTCAAGTTCACCTTCCTCTTCTGTGGCTCGATGATGGCGACGGGGAAATTACTTGTCTCGTACGCGCCGCCGGGGGCGGACCCGCCGAAAAAGCGAAAGGAGGCGATGCTTGGAACTCATGTCATCTGGGATATCGGTTTACAAAGCAGCTGCACCATGGTGGTGCCCTGGATCTCGAATACGACGTACCGCCAGACGATCGACGATTCCTTCACGGAGGGGGGCTACATCTCGGTCTTCTACCAAACGCGAATCGTCGTACCGCTCTCGACGCCGCGCGAAATGGACATTCTTGGTTTCGTATCGGCCTGCAATGACTTCTCGGTACGATTATTACGCGACACCACGCATATCGAACAAAAAGCGCTGGCGCAAGGACTTGGACAAATGCTGGAGTCGATGATCGACAACACGGTACGCGAAACGGTTGGTGCCGCGACGTCGCGCGACGCGTTACCGAATACGGAGGCGTCGGGGCCCACGCACTCGAAAGAAATACCGGCGCTCACGGCGGTCGAAACGGGCGCGACGAACCCGCTTGTTCCAAGCGATACCGTACAAACGCGACATGTTGTACAACATCGATCGCGAAGTGAATCGTCGATCGAAAGTTTTTTCGCGAGGGGGGCCTGTGTCACGATCATGACGGTCGACAACCCGGCGTCGACGACGAACAAAGATAAACTTTTCGCGGTCTGGAAGATCACGTACAAAGATACCGTACAATTACGACGAAAATTAGAATTCTTCACGTACTCGCGATTCGATATGGAGCTCACCTTTGTTGTTACCGCGAACTTCACGGAGACGAACAATGGACATGCGCTCAATCAAGTATATCAAATAATGTACGTACCGCCGGGGGCGCCGGTACCGGAGAAGTGGGATGACTACACGTGGCAGACGTCGTCGAACCCGTCGATCTTCTACACGTACGGTACCGCGCCGGCGCGAATATCGGTACCGTACGTGGGCATCTCGAACGCGTACTCGCACTTTTACGACGGCTTCTCGAAAGTTCCTTTAAAAGATCAATCGGCGGCGCTGGGGGACTCGTTATACGGGGCGGCGTCGCTCAACGATTTCGGTATTCTCGCGGTACGAGTAGTAAATGACCACAACCCGACGAAAGTCACGTCGAAAATTCGCGTATATTTAAAACCGAAACATATTCGCGTATGGTGCCCGCGGCCGCCGCGCGCGGTCGCGTATTACGGTCCTGGGGTCGATTATAAAGATGGTACGCTTACGCCGCTCTCGACGAAAGACCTCACGACGTAC"

back_to_original = ""
diff_total = 0
for i in range(0,len(recoded_sequence),3):
    if (recoded_sequence[i:i+3] != original_codons[i:i+3]):
        diff_total += 1
    back_to_original += codon_to_aa_dict[recoded_sequence[i:i+3]]
print("total number of codon diferences between original and recoded sequences " + str(diff_total))
if (original_AA == back_to_original):
    print("AA sequence remains the same, good")

print("coleman codon pair bias of the original sequence: " + str(get_cpb(original_codons)))
print("coleman codon pair bias of the recoded sequence: " + str(get_cpb(recoded_sequence)))
original_indep_score = get_cps_indep(original_codons)
original_frame1_score = get_cps(original_codons)
print("frame2&3 indep codon pair score of the original sequence: " + str(original_indep_score))
print("frame2&3 indep codon pair score of the recoded sequence: " + str(get_cps_indep(recoded_sequence)))

# now, for each amino acid, find all of the positions to scramble
for i in range(0,len(recoded_sequence),3):
    aa = codon_to_aa_dict[recoded_sequence[i:i+3]]
    aa_positions[aa].append(i)


recoded_codon_count_dict = get_codon_count_dict(recoded_sequence)

#print(aa_positions)
number_swaps = 10000

best_so_far_sequence = recoded_sequence
best_so_far_indep_cps = get_cps_indep(recoded_sequence)
best_so_far_frame1_cps = original_frame1_score
optimization_mode = 0 # 0 to bring indep_cps up, 
p = 0
while (p <= number_swaps):
    
    # pick an amino acid
    idx = random.randint(0, len(aa_to_syn_codons_dict.keys())-1)
    aa = list(aa_to_syn_codons_dict.keys())[idx]
    
    #print(aa_positions[aa])
    if (len(aa_positions[aa]) < 2):
        continue
    
    #print(len(aa_positions[aa]))
    
    # pick the position for the first codon
    pos1 = random.randint(0, len(aa_positions[aa])-1)
    pos2 = pos1
    while(pos2 == pos1):
        pos2 = random.randint(0, len(aa_positions[aa])-1)
    seq_pos1 = aa_positions[aa][pos1]
    seq_pos2 = aa_positions[aa][pos2]
    codon1 = best_so_far_sequence[seq_pos1:seq_pos1+3]
    codon2 = best_so_far_sequence[seq_pos2:seq_pos2+3]
    #print(codon1)
    #print(codon2)
    
    candidate_sequence = ""
    for i in range(0,len(original_codons),3):
        if (i == seq_pos1):
            candidate_sequence += codon2
        elif (i == seq_pos2):
            candidate_sequence += codon1
        else:
            candidate_sequence += best_so_far_sequence[i:i+3]
            
    #candidate_sequence = best_so_far_sequence[0:seq_pos1] + codon2 + best_so_far_sequence[seq_pos1+3:]
    #candicate_sequence = candidate_sequence[0:seq_pos2] + codon1 + candidate_sequence[seq_pos2+3:]
    #print(candidate_sequence)
    
    #print(candidate_sequence[seq_pos1:seq_pos1+3])
    #print(candidate_sequence[seq_pos2:seq_pos2+3])
    
    recoded_codon_count_dict = get_codon_count_dict(recoded_sequence)
    candidate_codon_count_dict = get_codon_count_dict(candidate_sequence)
    for codon in (recoded_codon_count_dict.keys()):
        if (recoded_codon_count_dict[codon] != candidate_codon_count_dict[codon]):
            print("recoded and candidate not equal for codon ", codon)
            print(str(recoded_codon_count_dict[codon]) + " " + str(candidate_codon_count_dict[codon]))
            break  
    
    candidate_indep_cps = get_cps_indep(candidate_sequence)
    candidate_frame1_cps = get_cps(candidate_sequence)
    # bring the independent cps back up to original
    if (optimization_mode == 0):
        if(candidate_indep_cps > best_so_far_indep_cps):
            if (candidate_indep_cps > original_indep_score):
                optimization_mode = 1
            else:
                best_so_far_sequence = candidate_sequence
                best_so_far_indep_cps = candidate_indep_cps
                best_so_far_frame1_cps = candidate_frame1_cps
                print("mode 0: " + str(best_so_far_indep_cps) + " " + str(candidate_frame1_cps))
                #print(get_cai(best_so_far_sequence))
    # now don't let the independent go down too much, but bring the frame1 cps down to original
    else:
        if ((candidate_indep_cps < 0) and (candidate_indep_cps > -32) and candidate_frame1_cps < best_so_far_frame1_cps):
            if (candidate_frame1_cps < original_frame1_score):
                break
            best_so_far_sequence = candidate_sequence
            best_so_far_indep_cps = candidate_indep_cps
            best_so_far_frame1_cps = candidate_frame1_cps
            print("mode 1: " + str(best_so_far_indep_cps) + " " + str(candidate_frame1_cps))
            #print(get_cai(best_so_far_sequence))
        
    p += 1

back_to_recoded = ""
diff_total = 0
for i in range(0,len(recoded_sequence),3):
    if (best_so_far_sequence[i:i+3] != recoded_sequence[i:i+3]):
        diff_total += 1
    back_to_recoded += codon_to_aa_dict[best_so_far_sequence[i:i+3]]
if (original_AA == back_to_recoded):
    print("AA sequence remains the same, good")
else:
    print(original_AA)
    print(back_to_recoded)

print("coleman codon pair bias of the candidate sequence: " + str(get_cpb(best_so_far_sequence)))
print("frame2&3 indep codon pair score of the candidate sequence: " + str(get_cps_indep(best_so_far_sequence)))
print(best_so_far_sequence)    
    

total number of codon diferences between original and recoded sequences 576
AA sequence remains the same, good
coleman codon pair bias of the original sequence: -0.03513233601841192
coleman codon pair bias of the recoded sequence: -0.036011507479861975
frame2&3 indep codon pair score of the original sequence: -31.61352552200005
frame2&3 indep codon pair score of the recoded sequence: -488.30513615799987
mode 0: -486.72490426299987 -30.190000000000055
0.567416802293
mode 0: -485.33558507699985 -28.81000000000003
0.567416802293
mode 0: -483.91223192799976 -28.528000000000027
0.567416802293
mode 0: -483.2732058479997 -28.70600000000003
0.567416802293
mode 0: -481.2927908829997 -30.558000000000035
0.567416802293
mode 0: -477.93523466799974 -28.937000000000012
0.567416802293
mode 0: -476.9442313539997 -29.624999999999993
0.567416802293
mode 0: -476.1627920829998 -29.592999999999996
0.567416802293
mode 0: -475.87154940399967 -30.23899999999999
0.567416802293
mode 0: -475.41771172499966 -33.3

0.567416802293
mode 0: -286.95123876300033 36.031
0.567416802293
mode 0: -286.54411204100035 35.019
0.567416802293
mode 0: -286.4896731860004 34.425999999999995
0.567416802293
mode 0: -286.2524765820004 34.248999999999974
0.567416802293
mode 0: -285.65709949200044 34.52099999999998
0.567416802293
mode 0: -284.96006621700036 34.32399999999998
0.567416802293
mode 0: -284.7747032630004 34.32199999999997
0.567416802293
mode 0: -283.6806275580004 35.769
0.567416802293
mode 0: -283.54965371100036 36.52599999999999
0.567416802293
mode 0: -282.9985036200004 35.899999999999984
0.567416802293
mode 0: -281.81908972800034 37.15399999999999
0.567416802293
mode 0: -281.33266364100024 38.02700000000001
0.567416802293
mode 0: -279.56772834000026 39.26
0.567416802293
mode 0: -279.31659650600034 39.42799999999999
0.567416802293
mode 0: -279.1955198180004 38.97199999999999
0.567416802293
mode 0: -278.1990367520004 40.60599999999999
0.567416802293
mode 0: -277.5468829710003 41.33699999999999
0.56741680229

mode 0: -185.6390982470003 76.46299999999997
0.567416802293
mode 0: -185.27496409600028 78.25299999999999
0.567416802293
mode 0: -185.11146820000025 77.412
0.567416802293
mode 0: -183.17985087300022 78.12299999999996
0.567416802293
mode 0: -182.8123219070002 78.28499999999998
0.567416802293
mode 0: -182.19321506700032 79.65900000000006
0.567416802293
mode 0: -182.1547791180003 80.40600000000005
0.567416802293
mode 0: -182.0949134250003 80.16200000000005
0.567416802293
mode 0: -182.00477389700023 81.05300000000007
0.567416802293
mode 0: -181.49786261900027 80.97000000000008
0.567416802293
mode 0: -180.48374078900034 81.59600000000009
0.567416802293
mode 0: -180.08697903000032 81.65000000000009
0.567416802293
mode 0: -180.06191229500033 80.20900000000003
0.567416802293
mode 0: -179.8758063220003 80.13000000000008
0.567416802293
mode 0: -179.64446493700035 80.54000000000006
0.567416802293
mode 0: -179.5910261220003 80.94400000000007
0.567416802293
mode 0: -179.38164943000027 80.2940000000

mode 0: -127.28648430800027 94.50800000000002
0.567416802293
mode 0: -126.96432640500025 95.349
0.567416802293
mode 0: -126.94316740200024 97.01599999999998
0.567416802293
mode 0: -126.32097308000024 97.10099999999998
0.567416802293
mode 0: -126.26057336200023 98.209
0.567416802293
mode 0: -125.87788137200019 98.50699999999999
0.567416802293
mode 0: -125.75102016200019 100.10300000000001
0.567416802293
mode 0: -125.55065534000018 99.20399999999998
0.567416802293
mode 0: -125.05616962600018 99.79799999999997
0.567416802293
mode 0: -124.7354680680002 99.46499999999999
0.567416802293
mode 0: -124.1161901970002 98.52799999999999
0.567416802293
mode 0: -123.7850557170002 99.48599999999999
0.567416802293
mode 0: -123.73885063700018 100.29899999999998
0.567416802293
mode 0: -123.69860181900022 100.35099999999998
0.567416802293
mode 0: -123.01031849900023 100.19699999999997
0.567416802293
mode 0: -122.26206089100025 99.98999999999997
0.567416802293
mode 0: -121.67742188100024 100.6759999999999

mode 0: -94.78603728300007 117.85300000000002
0.567416802293
mode 0: -93.95301472900007 118.72400000000003
0.567416802293
mode 0: -93.90456493000008 118.82700000000003
0.567416802293
mode 0: -93.79360728600007 117.89700000000002
0.567416802293
mode 0: -93.72030890400006 119.87100000000002
0.567416802293
mode 0: -93.66782806200004 121.515
0.567416802293
mode 0: -93.60731514300002 120.371
0.567416802293
mode 0: -93.53205440500005 120.41000000000001
0.567416802293
mode 0: -93.44762475800003 120.93700000000001
0.567416802293
mode 0: -93.39794855300002 120.056
0.567416802293
mode 0: -93.39335895900001 119.453
0.567416802293
mode 0: -93.10486169800005 118.84700000000001
0.567416802293
mode 0: -92.86014093000006 119.06500000000001
0.567416802293
mode 0: -92.69686263700004 120.40700000000002
0.567416802293
mode 0: -92.66560470100003 119.87900000000002
0.567416802293
mode 0: -92.63934315400003 119.44100000000003
0.567416802293
mode 0: -92.58827104600006 119.30300000000004
0.567416802293
mode 0:

In [74]:
print(get_cps(original_codons))
print(get_cps_indep(original_codons))

-30.52999999999996
-31.61352552200005


In [75]:
print(get_cps(recoded_sequence))
print(get_cps_indep(recoded_sequence))

-31.294000000000057
-488.30513615799987


In [97]:
print(get_cps(best_so_far_sequence))
print(get_cps_indep(best_so_far_sequence))

-30.22900000000001
-31.01385238599999


In [130]:
number = 100

p = 0

while(p <= number):
    print((number-p)/number)
    if (random.random() <= ((number-p)/number)):
        print("Accept")
    else:
        print("Reject")
    p += 1

1.0
Accept
0.99
Accept
0.98
Accept
0.97
Accept
0.96
Accept
0.95
Accept
0.94
Reject
0.93
Accept
0.92
Reject
0.91
Accept
0.9
Accept
0.89
Accept
0.88
Accept
0.87
Accept
0.86
Accept
0.85
Accept
0.84
Accept
0.83
Accept
0.82
Reject
0.81
Accept
0.8
Accept
0.79
Reject
0.78
Reject
0.77
Accept
0.76
Reject
0.75
Accept
0.74
Accept
0.73
Accept
0.72
Accept
0.71
Accept
0.7
Accept
0.69
Reject
0.68
Accept
0.67
Accept
0.66
Accept
0.65
Reject
0.64
Reject
0.63
Accept
0.62
Accept
0.61
Accept
0.6
Accept
0.59
Reject
0.58
Reject
0.57
Accept
0.56
Accept
0.55
Accept
0.54
Accept
0.53
Reject
0.52
Accept
0.51
Accept
0.5
Reject
0.49
Reject
0.48
Accept
0.47
Reject
0.46
Reject
0.45
Accept
0.44
Reject
0.43
Accept
0.42
Reject
0.41
Reject
0.4
Accept
0.39
Reject
0.38
Reject
0.37
Reject
0.36
Accept
0.35
Reject
0.34
Reject
0.33
Reject
0.32
Reject
0.31
Accept
0.3
Reject
0.29
Accept
0.28
Reject
0.27
Reject
0.26
Accept
0.25
Accept
0.24
Reject
0.23
Reject
0.22
Reject
0.21
Reject
0.2
Accept
0.19
Reject
0.18
Reject
0.17
Reject
0

In [131]:
print(len(cp_scores))

3721


In [132]:
print(len(cp_scores_indep))

3337


In [133]:
print(len(codon_CAI))

65


In [134]:
print(codon_CAI)

{'TAG': 0.44377118949722283, 'CGT': 0.3734363750317257, 'TTT': 0.8884718463567317, 'CGG': 0.9486069169847872, 'CCT': 0.914253748895514, 'ATA': 0.3744448559504284, 'TAT': 0.8292607258735172, 'ACG': 0.3252247560574498, 'TGG': 1.0, 'AGA': 1.0, 'CTA': 0.1833426195608562, 'TCC': 0.8855720603314994, 'AAG': 1.0, 'NNN': 0, 'AAA': 0.7931770345548678, 'GTC': 0.5131019513836897, 'GCA': 0.5919417792066642, 'AGG': 0.9567804994276421, 'AGT': 0.64739533894404, 'CCG': 0.35195120555834547, 'GCT': 0.6712730785793871, 'ACC': 1.0, 'CGC': 0.8347050932088822, 'CAC': 1.0, 'GAT': 0.9059877250022975, 'TCT': 0.7841126628210786, 'GAC': 1.0, 'TTG': 0.3328178529483908, 'TCA': 0.6443317788014337, 'GTG': 1.0, 'TTA': 0.20328228886286218, 'ACA': 0.8400394693564303, 'TTC': 1.0, 'CCC': 1.0, 'GGT': 0.49061024080954835, 'CGA': 0.519758003946939, 'GGG': 0.7310082226948681, 'AAT': 0.924839675927619, 'GAA': 0.7711047509307959, 'AAC': 1.0, 'CAT': 0.7444586656721225, 'TAC': 1.0, 'TGT': 0.8715803813263944, 'TAA': 0.564452138786

In [160]:
import itertools
all_nine_mers = [''.join(x) for x in itertools.product('ACGT', repeat=9)]
codon_info_dict = dict()
for codon in codon_CAI:
    codon_info_dict[codon] = dict()
    codon_info_dict[codon]["cai"] = codon_CAI[codon]
    codon_info_dict[codon]["occs"] = 0
    codon_info_dict[codon]["sum_23"] = 0.0
    codon_info_dict[codon]["score"] = -1
for ninemer in all_nine_mers:
    count += 1
    if ((ninemer[1:7] in cp_scores_indep) and (ninemer[2:8] in cp_scores_indep)):
        score = cp_scores_indep[ninemer[1:7]] + cp_scores_indep[ninemer[2:8]]
        if (ninemer == "ACGTACGTA"):
            print(ninemer)
            print(score)
            break
        for i in [0, 3, 6]:
            codon = ninemer[i:i+3]
            #print(codon)
            codon_info_dict[codon]["occs"] += 1
            codon_info_dict[codon]["sum_23"] += score    
for aa in (aa_to_syn_codons_dict.keys()):
    print(aa)
    for codon in (aa_to_syn_codons_dict[aa]):
        if (codon_info_dict[codon]["occs"]):
            codon_info_dict[codon]["score"] = codon_info_dict[codon]["sum_23"]/codon_info_dict[codon]["occs"]
            print(codon + " " + "{0:.2f}".format(codon_info_dict[codon]["cai"]) + " " + "{0:.2f}".format(codon_info_dict[codon]["score"]))

ACGTACGTA
-1.492299305
Y
TAC 1.00 -0.46
TAT 0.83 -0.39
M
ATG 1.00 -0.13
H
CAC 1.00 -0.06
CAT 0.74 -0.06
L
CTA 0.18 -0.18
CTC 0.48 -0.09
CTG 1.00 0.11
CTT 0.34 -0.03
TTA 0.20 -0.27
TTG 0.33 -0.17
K
AAA 0.79 -0.19
AAG 1.00 -0.08
E
GAA 0.77 -0.38
GAG 1.00 -0.13
N
AAC 1.00 -0.25
AAT 0.92 -0.27
*
TAA 0.56 -0.36
TAG 0.44 -0.36
TGA 1.00 -0.08
T
ACA 0.84 -0.09
ACC 1.00 -0.17
ACG 0.33 -0.51
ACT 0.74 -0.20
P
CCA 0.89 0.08
CCC 1.00 -0.10
CCG 0.35 -0.41
CCT 0.91 0.00
G
GGA 0.76 -0.16
GGC 1.00 -0.08
GGG 0.73 -0.17
GGT 0.49 -0.31
I
ATA 0.37 -0.39
ATC 1.00 -0.40
ATT 0.80 -0.33
V
GTA 0.27 -0.40
GTC 0.51 -0.31
GTG 1.00 -0.06
GTT 0.41 -0.29
C
TGC 1.00 -0.04
TGT 0.87 -0.03
F
TTC 1.00 -0.16
TTT 0.89 -0.17
R
AGA 1.00 0.12
AGG 0.96 0.07
CGA 0.52 -0.79
CGC 0.83 -0.59
CGG 0.95 -0.57
CGT 0.37 -0.58
S
AGC 1.00 0.03
AGT 0.65 0.02
TCA 0.64 -0.08
TCC 0.89 -0.16
TCG 0.22 -0.53
TCT 0.78 -0.11
D
GAC 1.00 -0.37
GAT 0.91 -0.51
Q
CAA 0.37 -0.09
CAG 1.00 0.14
W
TGG 1.00 0.10
A
GCA 0.59 -0.16
GCC 1.00 -0.06
GCG 0.26 -0.51

In [139]:
print(len(all_nine_mers))


262144


In [156]:
print(cp_scores_indep["CGTACG"])
print(cp_scores_indep["GTACGT"])

-0.900977206
-0.591322099


In [153]:
print(-0.527626688-0.527626688)

-1.055253376


In [158]:
print(sum(cp_scores_indep.values())/len(cp_scores_indep))

-0.10149142491459401
